In [1]:
import networkx as nx
from node2vec import Node2Vec

In [2]:
# Create a graph
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

In [5]:
graph.adj

AdjacencyView({0: {1: {}, 2: {}, 7: {}, 8: {}, 10: {}, 13: {}, 15: {}, 19: {}, 20: {}, 22: {}, 23: {}, 24: {}, 27: {}, 28: {}, 30: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 41: {}, 42: {}, 46: {}, 51: {}, 53: {}, 54: {}, 55: {}, 58: {}, 59: {}, 60: {}, 61: {}, 64: {}, 65: {}, 66: {}, 67: {}, 68: {}, 72: {}, 73: {}, 75: {}, 80: {}, 82: {}, 85: {}, 86: {}, 88: {}, 89: {}, 91: {}, 93: {}, 95: {}, 96: {}, 98: {}, 99: {}}, 1: {0: {}, 2: {}, 3: {}, 5: {}, 6: {}, 7: {}, 10: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 18: {}, 19: {}, 20: {}, 23: {}, 26: {}, 28: {}, 29: {}, 33: {}, 34: {}, 35: {}, 37: {}, 38: {}, 41: {}, 42: {}, 48: {}, 50: {}, 52: {}, 54: {}, 56: {}, 57: {}, 66: {}, 68: {}, 69: {}, 70: {}, 72: {}, 74: {}, 75: {}, 76: {}, 77: {}, 80: {}, 81: {}, 83: {}, 84: {}, 86: {}, 93: {}, 98: {}, 99: {}}, 2: {0: {}, 1: {}, 3: {}, 5: {}, 6: {}, 8: {}, 10: {}, 11: {}, 12: {}, 15: {}, 16: {}, 18: {}, 19: {}, 24: {}, 26: {}, 28: {}, 29: {}, 30: {}, 31: {}, 36: {}, 37: {}, 39:

In [ ]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)  # Use temp_folder for big graphs

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

# Look for most similar nodes
model.wv.most_similar('2')  # Output node names are always strings

# Save embeddings for later use
model.wv.save_word2vec_format(EMBEDDING_FILENAME)

# Save model for later use
model.save(EMBEDDING_MODEL_FILENAME)

# Embed edges using Hadamard method
from node2vec.edges import HadamardEmbedder

edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

# Look for embeddings on the fly - here we pass normal tuples
edges_embs[('1', '2')]
''' OUTPUT
array([ 5.75068220e-03, -1.10937878e-02,  3.76693785e-01,  2.69105062e-02,
       ... ... ....
       ..................................................................],
      dtype=float32)
'''

# Get all edges in a separate KeyedVectors instance - use with caution could be huge for big networks
edges_kv = edges_embs.as_keyed_vectors()

# Look for most similar edges - this time tuples must be sorted and as str
edges_kv.most_similar(str(('1', '2')))

# Save embeddings for later use
edges_kv.save_word2vec_format(EDGES_EMBEDDING_FILENAME)
